In [1]:
import numpy as np
import numpy.linalg as npla
import scipy
import scipy.linalg as sla
import pickle
import os
from sklearn.datasets import load_svmlight_file

from methods import GradientDescent, Newton, Nesterov, RBFGS, LBFGS, BFGS
from methods.rbfgs import Uniform, Gaussian, CustomDiscrete
from oracles import LogRegL2Oracle, create_log_reg_oracle, QuadraticOracle
from utils import select_basis_columns, run_rbfgs_experiment, run_all_methods

In [2]:
file = './datasets/a9a.txt'
A, b = load_svmlight_file(file, n_features=None)
print('(n, d) = {}'.format(A.shape))
print('density = {}'.format(A.count_nonzero() / (A.shape[0] * A.shape[1])))

(n, d) = (29159, 123)
density = 0.11275939571014765


In [21]:
with open('./results/a9a/regcoef=1.00e-03/svd.pkl', 'rb') as file:
    U, sigma_diag, V = pickle.load(file)
print('Smoothness based on singular values: {}'.format(sigma_diag.max()**2 / (4. * A.shape[0])))
print('Smoothness based on row norms: {}'.format(np.max(npla.norm(A.toarray(), axis=1))**2 / 4.))

Smoothness based on singular values: 1.5720020336101925
Smoothness based on row norms: 3.5


In [4]:
sketch_sizes = [1, 2, 3, 5, 10, 30, 50, 100, 123]
def experiment_a9a(regcoef):
    oracle = create_log_reg_oracle(A, b, regcoef)
    run_all_methods(
        oracle=oracle, 
        sketch_sizes=sketch_sizes, 
        max_iter=1000, 
        mat=A.toarray(), 
        output_folder='./results/a9a/regcoef={:.2e}'.format(regcoef), 
        sigma_tolerance=1e-8, 
        method_tolerance=1e-16, 
        stopping_criteria='grad_abs', 
        random_state=0
    )

In [ ]:
%%time
experiment_a9a(1e-2)

Read SVD from ./results/a9a/regcoef=1.00e-02/svd.pkl
Singular values above tolerance: 108

RBFGS-SVD sketch... Done
RBFGS-SVD sketch no sigma... 

In [ ]:
%%time
experiment_a9a(1e-3)

In [ ]:
%%time
experiment_a9a(1e-4)

In [ ]:
%%time
experiment_a9a(1e-5)

### Nesterov

In [3]:
file = './datasets/a9a.txt'
A, b = load_svmlight_file(file, n_features=None)
regcoef = 1e-3
print('(n, d) = {}'.format(A.shape))
print('density = {}'.format(A.count_nonzero() / (A.shape[0] * A.shape[1])))
oracle = create_log_reg_oracle(A, b, regcoef=regcoef)

(n, d) = (29159, 123)
density = 0.11275939571014765


In [4]:
np.random.seed(42)
x_0 = np.random.normal(size=A.shape[1])
output_folder = './results/a9a/regcoef=1.00e-03'
method = Nesterov(oracle, x_0, stopping_criteria='grad_abs', 
                  tolerance=1e-16)
method.run(max_iter=50000, max_time=30)
method.oracle = None
with open('{}/nesterov.pkl'.format(output_folder), 'wb') as file:
    pickle.dump(method, file)